# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else...
4. You must follow the Programming Requirments for this course.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

<div style="color: white; background: #C83F49; margin:20px; padding: 20px;">
    <strong>Academic Integrity and Copyright</strong>
    <p>You are not permitted to consult outside sources (Stackoverflow, YouTube, ChatGPT, etc.) or use "code assistance" (Co-Pilot, etc) to complete this assignment. By submitting this assignment for grading, you certify that the submission is 100% your own work, based on course materials, group interactions, instructor guidance. You agree to comply by the requirements set forth in the Syllabus, including, by reference, the JHU KSAS/WSE Graduate Academic Misconduct Policy.</p>
    <p>Sharing this assignment either directly (e.g., email, github, homework site) or indirectly (e.g., ChatGPT, machine learning platform) is a violation of the copyright. Additionally, all such sharing is a violation the Graduate Academic Misconduct Policy (facilitating academic dishonesty is itself academic dishonesty), even after you graduate.</p>
    <p>If you have questions or if you're unsure about the policy, ask via Canvas Inbox. In this case, being forgiven is <strong>not</strong> easier than getting permission and ignorance is not an excuse.</p>
    <p>This assignment is copyright (&copy Johns Hopkins University &amp; Stephyn G. W. Butcher). All rights reserved.</p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [2]:
full_world = [
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '⛰', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌋', '🌋', '🌋', '🌋', '🌾', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌋', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌋', '🌋', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾'], 
    ['🌲', '🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌋', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾'], 
    ['🌲', '🌾', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌋', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌾', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🌾', '⛰', '🌾', '🌾', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌋', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '🌾', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰'], 
    ['🌾', '🌾', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰'], 
    ['🌾', '🌾', '🌋', '🌲', '🌲', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🐊', '🌾', '🐊', '🐊', '🌾', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌲', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌋', '🌾'], 
    ['🌾', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾']
]

Note: emojis are not fixed width so here is a function that will print out a List of Lists of emojis in an HTML table.

In [3]:
from IPython.display import display_html

def display_emoji_grid(emoji_grid):
    """
    Display a List of Lists of emojis in a perfect grid (table) in a Jupyter Notebook.
    
    Parameters:
    emoji_grid (list of list of str): A 2D list containing emojis to display in a grid.
    """
    # Create HTML table
    html = '<table style="border-collapse: collapse;">'
    
    for row in emoji_grid:
        html += '<tr>'
        for emoji in row:
            html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{emoji}</td>'
        html += '</tr>'
    
    html += '</table>'
    
    # Display the HTML table
    display_html(html, raw=True)
#

In [4]:
display_emoji_grid(full_world)

🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,🌾,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [5]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [6]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [7]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!
* Use the provided `display_emoji_grid` function.

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [8]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic

`heuristic` takes in the current state information and returns a cost based on Manhattan distance and scaled by the cost of the terrain. **Used by**: [a_star_search](#a_star_search)
* **state** Tuple[int, int]: current state.
* **goal** Tuple[int, int]: goal state.
* **world** List[List[str]]: map of the world represented by emojis.
* **costs** Dict[str, int]: costs of traversing each terrain.

**returns** Int: calculated heruistic cost.


In [84]:
def heuristic( state: Tuple[int, int], goal: Tuple[int, int], world: List[List[str]], costs: Dict[str, int] ): # you can add formal parameters
    terrain = world[state[0]][state[1]]
    if terrain not in costs:
        return 99999
    return (abs(state[0] - goal[0]) + abs(state[1] - goal[1])) * costs[terrain]

In [85]:
# assertions/unit tests
assert heuristic((0,0), (0,0), full_world, COSTS) == 0
assert heuristic((1,1), (4,4), full_world, COSTS) == 6
assert heuristic((4,2), (4,4), full_world, COSTS) == 99999


## in_bound
`in_bound` Determines if the current state is a valid position on the map. **Used by**: [a_star_search](#a_star_search)
* **state** Tuple[int, int]: current state.
* **world** List[List[str]]: map of the world represented by emojis.

**returns** Bool: validity of the state.


In [86]:
def in_bound(state: Tuple[int, int], world: List[List[str]]) -> bool:
    return 0 <= state[0] < len(world) and 0 <= state[1] < len(world[0])

In [139]:
assert in_bound((0,0), full_world) == True
assert in_bound((-1,1), full_world) == False
assert in_bound((1,-1), full_world) == False

## construct_path
`construct_path` Formats the shortest path to the goal state. **Used by**: [a_star_search](#a_star_search)
* **path** Dict[Tuple[int, int], Tuple[int, int]]: inverse path taken by the search algorithm.
* **current_state** Tuple[int, int]: ending goal state.


**returns** List[Tuple[int, int]]: formatted path from start to goal.

In [172]:
def construct_path(path: Dict[Tuple[int, int], Tuple[int, int]], current_state: Tuple[int, int]) -> List[Tuple[int, int]]:
    reconstructed = []
    while current_state is not None:
        reconstructed.append(current_state)
        current_state = path[current_state]
    reconstructed.reverse()
    return reconstructed

In [173]:
assert construct_path({(0,0): None, (1,0): (0,0), (1,1): (1,0)}, (1,1)) == [(0,0), (1,0), (1,1)]
assert construct_path({(0,0): None, (1,0): (0,0), (1,1): (1,0)}, (0,0)) == [(0,0)]
assert construct_path({(1,2): None, (1,3): (1,2), (2,3): (1,3)}, (2,3)) == [(1,2), (1,3), (2,3)]

<a id="a_star_search"></a>
## a_star_search

`a_star_search` implements the A* search algorithm is optimized for pathfinding through a graph by considering both the actual cost and estimated cost of each transition. By implementing the frontier as a priority queue to keep track of the next state to visit, the algorithm is able to minimize the cost of the taken path. **Used by**: [Problems](#problems)

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [ ]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier = []
    explored = set()
    start_node = (start, 0, heuristic(start, goal, world, costs))  # (state, cost, heuristic)
    frontier.append(start_node)
    path = {start: None}
    scores = {start: 0}

    while frontier:
        frontier.sort(key=lambda x: x[1] + x[2])
        current_node = frontier.pop(0)
        current_state, current_cost, _ = current_node

        if current_state == goal:
            return construct_path(path, current_state)

        explored.add(current_state)

        for move in moves:
            next_state = (current_state[0] + move[0], current_state[1] + move[1])
            
            if not in_bound(next_state, world):
                continue
            
            terrain = world[next_state[0]][next_state[1]]
            if terrain not in costs or next_state in explored:
                continue
            
            h = heuristic(next_state, goal, world, costs)
            g = current_cost + costs[terrain]
            f = g + h

            if next_state not in scores or f < scores[next_state]:
                scores[next_state] = f
                frontier.append((next_state, g, h))
                path[next_state] = current_state

    return []

## pretty_print_path
`pretty_print_path` Prints the map in a formated html table, where the path states are represented by directional emojis. The cost of the path is calculated and returned. **Used by**: [Problems](#Problems)


* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.

In [163]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    html = '<table style="border-collapse: collapse;">'
    cost = 0
    for i in range(len(world)):
        for j in range(len(world[i])):
            if (i, j) in path:
                terrain = world[i][j]
                cost += costs[terrain]
                index = path.index((i, j))
                if (i, j) == goal:
                    emoji = '🎁'
                else:
                    if i + 1 == path[index + 1][0] and j == path[index + 1][1]:
                        emoji = '⏬'
                    elif i - 1 == path[index + 1][0] and j == path[index + 1][1]:
                        emoji = '⏫'
                    elif i == path[index + 1][0] and j + 1 == path[index + 1][1]:
                        emoji = '⏩'
                    elif i == path[index + 1][0] and j - 1 == path[index + 1][1]:
                        emoji = '⏪'
                    else:
                        emoji = '❓'

                html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{emoji}</td>'
            else:
                html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{world[i][j]}</td>'
        html += '</tr>'
    html += '</table>'
    
    display_html(html, raw=True)
    return cost


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [164]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬ 🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 ⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 🎁

total path cost: 13
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [165]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬ 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🐊 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 ⛰ ⛰ ⛰ ⏬ 🌾 🌾 🌾 🌾 ⛰ ⛰ 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 ⛰ 🌋 🌋 ⛰ ⏬ 🌾 🌾 🌾 🌾 ⛰ ⛰ ⛰ 🌾 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🌋 🌾 🌾 🌾 ⛰ 🌾 🌾 ⛰ ⛰ 🌋 🌾 ⏬ 🌾 🌾 🌾 ⛰ ⛰ 🌋 ⛰ ⛰ 🐊 🐊 🐊 🐊 🐊 🌾 🌋 🌋 🌋 🌋 🌾 ⛰ 🌾 🌾 🌾 ⛰ 🌋 🌾 ⏩ ⏬ 🌋 ⛰ ⛰ 🌋 🌋 ⛰ ⛰ 🐊 🐊 🐊 🐊 🐊 🌋 🌋 🌲 🌋 🌋 🌋 ⛰ ⛰ 🌾 🌾 ⛰ ⛰ 🌾 🌲 ⏬ 🌋 🌋 🌋 🌋 ⛰ ⛰ ⛰ 🐊 🐊 🐊 🌾 🌾 🌾 🌋 🌲 🌲 🌋 🌋 ⛰ ⛰ 🌾 ⛰ ⛰ 🌾 🌾 🌲 ⏬ 🌲 🌋 🌋 ⛰ ⛰ ⛰ 🌾 🌾 🐊 ⏩ ⏩ ⏩ ⏬ 🌲 🌲 🌲 🌲 🌋 🌋 ⛰ ⛰ ⛰ 🌾 🌾 🌾 🌲 ⏬ 🌲 🌋 🌲 ⛰ 🌾 🌾 ⏩ ⏩ ⏩ ⏫ ⛰ ⛰ ⏬ 🌲 🌲 🌲 🌲 🌲 🌋 🌋 ⛰ ⛰ 🌾 🌾 🌾 🌲 ⏬ 🌲 🌲 🌲 🌾 🌾 🌾 ⏫ ⛰ ⛰ ⛰ ⛰ 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 🌋 ⛰ ⛰ 🌾 🌾 🌾 🌲 ⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏫ ⛰ ⛰ 🌋 🌋 🌲 ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 🌋 🌋 ⛰ 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌾 🌾 🌾 🌾 🌾 ⛰ 🌋 🌋 🌋 ⛰ ⏩ ⏬ 🌲 🌲 🌲 🌲 🌲 🌲 🌋 🌾 🌾 ⛰ 🌾 🌲 🌲 🌲 🌲 🐊 🌾 ⛰ 🌾 🌾 🌋 🌋 ⛰ ⛰ 🌾 ⛰ ⏩ ⏬ 🌲 🌲 🌲 🌲 🌲 🌋 🌾 🌋 ⛰ 🌾 🌲 🌲 🌲 🐊 🐊 🐊 🌋 🌾 ⛰ 🌋 🌋 🌾 🌾 🌾 ⛰ 🌋 ⏬ 🌲 🌲 🌲 🌲 🌋 🌋 🌾 🌋 🌋 ⛰ 🌲 🌲 🌲 🐊 🐊 🐊 🌋 ⛰ ⛰ 🌋 ⛰ 🌾 🐊 🌾 ⛰ 🌋 ⏩ ⏬ 🌲 🌾 🌾 🌋 🌾 🌾 🌋 🌋 ⛰ 🌲 🌲 🌲 🌲 🐊 🐊 🌋 🌋 🌋 🌋 🌾 🌾 🐊 🌾 ⛰ 🌋 🌋 ⏩ ⏬ 🌾 🌋 🌾 🌾 🌾 ⛰ 🌋 ⛰ 🌾 🌲 🌲 🌲 🌲 🐊 🐊 🌋 🌋 🌾 🌾 🌾 🐊 🐊 🌾 ⛰ 🌋 🌲 ⏬ 🌾 🌾 🌾 🌾 ⛰ ⛰ 🌋 ⛰ 🌾 🌾 🌲 🌲 🌲 🐊 🐊 🌋 🌾 🌾 🌾 🐊 🐊 🐊 🐊 ⛰ 🌋 🌋 ⏩ ⏬ 🌾 🌾 🌾 ⛰ 🌋 ⛰ ⛰ 🌾 🌾 🌋 🌲 🌲 🌾 🐊 🐊 🐊 🌾 🌾 🐊 🌾 🐊 🐊 🌾 🌾 🌋 ⛰ ⏬ 🌾 🌾 ⛰ 🌋 🌋 ⛰ 🌾 🌾 🌋 🌋 🌲 🌾 🌾 🌾 🐊 🐊 🐊 🌾 🌾 🌾 🐊 🐊 🐊 🌾 🌋 ⛰ ⏬ 🌾 🌾 ⛰ 🌋 🌾 ⛰ 🌾 🌾 🌋 🌋 🌾 🌾 🌾 🌾 🐊 🌾 🌾 ⛰ 🌾 🌾 🌾 🌾 🌾 🌋 🌋 🌾 ⏩ ⏩ ⏩ ⏬ ⛰ 🌋 ⛰ 🌾 🌾 🌋 ⛰ 🌾 🌾 🌾 🌾 🌾 🌾 ⛰ 🌋 ⛰ ⛰ 🌾 🌾 ⛰ 🌋 🌾 🌾 🌾 🐊 🐊 ⏩ ⏬ 🌋 🌋 🌾 🌾 🌋 ⛰ ⛰ ⛰ 🌾 🌾 🌾 ⛰ ⛰ 🌋 🌋 🌋 ⛰ ⛰ 🌋 🌋 🌾 🌾 🌾 🐊 🐊 🐊 ⏩ ⏬ 🌋 ⛰ 🌾 🌋 ⛰ ⛰ 🌋 ⛰ 🌾 ⛰ ⛰ ⛰ 🌋 🌋 ⛰ 🌾 🌋 🌋 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 ⏬ 🌋 ⛰ 🌾 🌋 🌋 🌋 🌋 🌋 ⛰ ⛰ ⛰ 🌾 ⛰ ⛰ 🌾 🌾 ⛰ ⛰ 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 ⏩ ⏩ ⏬ 🌾 🌋 🌋 🌋 🌋 ⛰ 🌾 ⛰ 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🐊 🐊 ⏬ 🌾 🌾 ⛰ ⛰ ⛰ 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🐊 🐊 ⏬ 🌾 🌾 ⛰ 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🐊 🐊 🎁

total path cost: 119
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (8, 8), (7, 8), (7, 9), (7, 10), (7, 11), (6, 11), (6, 12), (6, 13), (6, 14), (7, 14), (8, 14), (9, 14), (10, 14), (10, 15), (11, 15), (11, 16), (12, 16), (13, 16), (13, 17), (14, 17), (14, 18), (15, 18), (16, 18), (16, 19), (17, 19), (18, 19), (19, 19), (19, 20), (19, 21), (19, 22), (20, 22), (20, 23), (21, 23), (21, 24), (22, 24), (23, 24), (23, 25), (23, 26), (24, 26), (25, 26), (26, 26)]


## Comments

(This is the place to leave me comments)

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.